In [91]:
import vectorbt as vbt
import yfinance
import pandas as pd
import numpy as np
import datetime

In [92]:
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days = 2)

In [93]:
btc_price = vbt.YFData.download(
    ['BTC-USD','ETH-USD'],
    interval = '1m',
    start = start_date,
    end = end_date,
    missing_index = True
).get('Close')

In [94]:
def custom_indicator(close, ma_window=14,rsi_window=25,entry = 30, exit = 70):
    close_5m = close.resample('5T').last()
    rsi = vbt.RSI.run(close,window = rsi_window).rsi
    rsi, _ = rsi.align(
        close,
        broadcast_axis = 0,
        method = 'ffill',
        join = 'right'
        )
    close = close.to_numpy()
    rsi = rsi.to_numpy()
    ma = vbt.MA.run(close, ma_window).ma.to_numpy()
    trend = np.where(rsi > entry, -1, 0)
    trend = np.where((rsi < exit) & (close < ma), 1, trend)
    return trend
    

In [95]:
ind = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    output_names=['value'],
    param_names=['ma_window','rsi_window','entry','exit']
    ).from_apply_func(
            custom_indicator,
            rsi_window = 14,
            ma_window = 50, 
            entry = 30,
            exit = 70,
            keep_pd = True
            )

In [96]:
res = ind.run(
    btc_price,
    rsi_window=[14,21,50],
    ma_window = [21,50,100],
    entry = [30,40],
    exit = [70,80],
    param_product = True
)

In [97]:
entries = res.value == 1.0
exits = res.value == -1.0

In [98]:
pf = vbt.Portfolio.from_signals(btc_price,entries,exits)

In [99]:
print(pf.total_return().to_string())

comb_ma_window  comb_rsi_window  comb_entry  comb_exit  symbol 
21              14               30          70         BTC-USD   -0.024799
                                                        ETH-USD   -0.048212
                                             80         BTC-USD   -0.024799
                                                        ETH-USD   -0.049734
                                 40          70         BTC-USD   -0.024667
                                                        ETH-USD   -0.045870
                                             80         BTC-USD   -0.024667
                                                        ETH-USD   -0.047396
                21               30          70         BTC-USD   -0.024944
                                                        ETH-USD   -0.049564
                                             80         BTC-USD   -0.024727
                                                        ETH-USD   -0.049564
                        

In [100]:
# max value of returns
print(pf.total_return().max())
# params for max returns
print(pf.total_return().idxmax())

-0.006101909261361413
(100, 14, 40, 70, 'BTC-USD')


In [101]:
returns = pf.total_return()

In [103]:
## to get a particular input returns (here --- ETH-USD)
returns = returns[returns.index.isin(['ETH-USD'],level = 'symbol')]
# max value of returns
print(returns.max())
# params for max returns
print(returns.idxmax())

-0.032729031815398456
(100, 14, 40, 80, 'ETH-USD')


###### ----------------

###### ----------------

In [ ]:
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days = 2)

In [ ]:
btc_price = vbt.YFData.download(
    ['BTC-USD','ETH-USD'],
    interval = '1m',
    start = start_date,
    end = end_date,
    missing_index = True
).get('Close')

In [ ]:
def custom_indicator(close, ma_window=14,rsi_window=25,entry = 30, exit = 70):
    close_5m = close.resample('5T').last()
    rsi = vbt.RSI.run(close,window = rsi_window).rsi
    rsi, _ = rsi.align(
        close,
        broadcast_axis = 0,
        method = 'ffill',
        join = 'right'
        )
    close = close.to_numpy()
    rsi = rsi.to_numpy()
    ma = vbt.MA.run(close, ma_window).ma.to_numpy()
    trend = np.where(rsi > entry, -1, 0)
    trend = np.where((rsi < exit) & (close < ma), 1, trend)
    return trend

In [ ]:
ind = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    output_names=['value'],
    param_names=['ma_window','rsi_window','entry','exit']
    ).from_apply_func(
            custom_indicator,
            rsi_window = 14,
            ma_window = 50, 
            entry = 30,
            exit = 70,
            keep_pd = True
            )

In [109]:
res = ind.run(
    btc_price,
    rsi_window = np.arange(10, 40, step=3, dtype=int) ,
    ma_window = np.arange(20, 200, step=15, dtype=int) ,
    entry = np.arange(10, 40, step=4, dtype=int) ,
    exit = np.arange(60, 85, step=4, dtype=int) ,
    param_product = True
)

In [110]:
entries = res.value == 1.0
exits = res.value == -1.0

In [111]:
pf = vbt.Portfolio.from_signals(btc_price,entries,exits)

In [128]:
returns = pf.total_return()

In [113]:
print(returns.to_string())

comb_ma_window  comb_rsi_window  comb_entry  comb_exit  symbol 
20              10               10          60         BTC-USD   -0.029740
                                                        ETH-USD   -0.049654
                                             64         BTC-USD   -0.029920
                                                        ETH-USD   -0.052476
                                             68         BTC-USD   -0.028940
                                                        ETH-USD   -0.050636
                                             72         BTC-USD   -0.027925
                                                        ETH-USD   -0.050906
                                             76         BTC-USD   -0.027147
                                                        ETH-USD   -0.051114
                                             80         BTC-USD   -0.028460
                                                        ETH-USD   -0.052481
                        

In [114]:
# max value of returns
print(returns.max())
# params for max returns
print(returns.idxmax())

-0.004398480560554248
(125, 16, 30, 64, 'BTC-USD')


In [129]:
## to get a particular input returns (here --- ETH-USD)
returns = returns[returns.index.isin(['ETH-USD'],level = 'symbol')]
# max value of returns
print(returns.max())
# params for max returns
print(returns.idxmax())

-0.027056213168829685
(155, 16, 10, 68, 'ETH-USD')


In [130]:
fig = returns.vbt.heatmap(
        x_level = 'comb_ma_window',
        y_level = 'comb_rsi_window'
        )
fig.show()

In [131]:
fig = returns.vbt.heatmap(
        x_level = 'comb_ma_window',
        y_level = 'comb_entry'
        )
fig.show()

In [132]:
returns = pf.total_return()

In [133]:
fig = returns.vbt.heatmap(
        x_level = 'comb_ma_window',
        y_level = 'comb_entry',
        slider_level = 'symbol',
        )
fig.show()

In [137]:
returns = pf.total_return()
returns = returns.groupby(level=['comb_exit','comb_entry','symbol']).mean()

In [138]:
fig = returns.vbt.heatmap(
        x_level = 'comb_exit',
        y_level = 'comb_entry',
        slider_level = 'symbol',
        )
fig.show()

In [140]:
returns = pf.total_return()
fig = returns.vbt.volume(
        x_level = 'comb_ma_window',
        y_level = 'comb_entry',
        z_level = 'comb_exit',
        slider_level = 'symbol',
        )
fig.show()